In [1]:
# 구글 드라이브 마운트(cjyjob1993@gmail.com)
from google.colab import drive
drive.mount('/content/drive')

# lib 디렉토리를을 환경 변수에 추가
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/myCode/lib')

Mounted at /content/drive


# 신경망 구현하기

In [2]:
# 라이브러리 임포트
import numpy as np
# 커스텀 라이브러리 임포트
from func_debug_log import debug

In [3]:
# debug_flag 설정(0:미출력, 1:출력)
debug_flag = 0

## 신경망 초기 상태 설정
weight, bias 초기화(랜덤값)

In [4]:
# 신경망 초기화 함수
#   params
#       없음
#   process
#       weights, bias이 랜덤하게 초기화된 net 생성
#   returns
#       초기화된 net (딕셔너리)
def init_network() :
    if debug_flag == 1 : debug('신경망 초기화', __name__)
    np.random.seed(42)
    net = {} 
    net['w1'] = np.random.rand(3, 4)
    net['b1'] = np.random.rand(4)
    net['w2'] = np.random.rand(4,2)
    net['b2'] = np.random.rand(2)
    return net

## 활성화 함수 구현
입력값의 시그모이드 함수의 결과값 

In [5]:
# 시그모이드 함수
#   params
#       x : 이전 층으로부터 전달 받은 가중합
#   process
#       x를 시그모이드 함수에 입력
#   returns
#       x가 시그모이드를 통과한 값
def sigmoid(x):
    if debug_flag == 1 : debug('시그모이드 함수 실행', __name__)
    z = 1/(1+np.exp(-x))
    return z

## 순전파
weight, bias 의 가중합 / 활성화 함수를 사용하여 입력층에서 은닉층을 거쳐, 출력층까지 계산.

### 입력층
데이터셋이 입력되는 층으로 features 수 만큼의 입력 노드를 갖는다.

### 은닉층
입력 신호를 가중합(가중치, 편향)하여 처리하는 층으로, 사용자가 볼 수 없으며, 노드수는 자유롭게 결정 가능하다. 2개 이상의 은닉층을 가진 신경망을 `딥러닝`이라고 한다.

### 출력층
연산을 마친 값이 출력되는 층이다. 문제의 종류에 따라, 노드 수, 활성화 함수가 결정된다.

#### 이진 분류
활성화 함수는 시그모이드, 노드 수 1개, 0~1 사이의 확률값이 출력

#### 다중 분류
활성화 함수는 소프트맥스, 노드 수는 레이블릐 클래스 수와 동일. 각 클래스일 확률을 0~1 사이 값으로 출력하며, 합은 1이 된다.

#### 회귀
활성화 함수는 사용하지 않으며, 출력층의 노드 수는 출력값의 특성 수와 동일.

In [10]:
# 순전파 함수
#   params
#       net : 학습시킬 신경망
#       x : 입력층에서 입력 받은 값
#   process
#       x를 은닉층(가중합) -> 시그모이드 함수 -> 출력층(가중합) 통과
#   returns
#       입력 x가 신경망을 통과해 산출된 값
def forward(net, x):
    if debug_flag == 1 : debug('순전파 함수 실행', __name__)
    w1 = net['w1']
    w2 = net['w2']
    b1 = net['b1']
    b2 = net['b2']

    # 은닉층
    h = np.dot(x, w1) + b1

    # 시그모이드 함수
    s = sigmoid(h)

    # 출력층
    o = np.dot(s, w2) + b2

    return o

In [7]:
if debug_flag == 1 : debug('신경망 인스턴스 생성', __name__)
net = init_network()

In [8]:
if debug_flag == 1 : debug('샘플 데이터 생성', __name__)
x = np.random.rand(3)

In [11]:
if debug_flag == 1 : debug('샘플데이터로, 신경망 순전파 진행', __name__)
y = forward(net, x)
print(y)

[1.59720489 1.78032042]
